# Fake unlimited dimensions

We would like to be able to manage time series of, say, radar data acquisitions.
The basic setup here would be a three dimension array with one unlimited dimension used for the timestamp.

We will try to fake it with a very large dimension for the the first axis, with the assumption that tiledb is  being lazy in the actual space allocation

In [ ]:
import numpy as np
import tiledb

In [ ]:
ctx = tiledb.Ctx()

We are defining a time dimension with a very large upper limit (10 years of acquisitions) with a tiling that correspond to a hour of acquisitions. 

In [ ]:
tdim = tiledb.Dim(ctx, 'time', domain=(0, 60*24*365*10), dtype=np.int32, tile=60)
xdim = tiledb.Dim(ctx, 'X', domain=(0, 1023), dtype=np.int32, tile=4)
ydim = tiledb.Dim(ctx, 'Y', domain=(0, 1023), dtype=np.int32, tile=5)

In [ ]:
dom = tiledb.Domain(ctx, tdim, xdim, ydim)

In [ ]:
dom.shape

In [ ]:
p = tiledb.Attr(ctx, name='precipitation', dtype=np.float32)

In [ ]:
schema = tiledb.ArraySchema(ctx, domain=dom, sparse=False, attrs=[p])

In [ ]:
aname = 'radar01'

In [ ]:
tiledb.DenseArray.create(aname, schema)

In [ ]:
data = np.zeros((1024, 1024), dtype=np.float32)

Now we insert a small number of images

In [ ]:
with tiledb.DenseArray(ctx, aname, mode='w') as A:
    for i in range(8):
        A[i:(i+1), :, :] = data

Each insertion is managed by tiledb as a single data fragment. Each fragment is contained in subfolder of the main array folder used by tiledb to manage the data. 

In [ ]:
!ls -l radar01

Of course, this could become a major problem, since it would generate a new data fragment for each data write. TileDB has a meccanism, called consolidation, that will collapse fragments together. 

In [ ]:
# tiledb.consolidate(ctx, aname)

Unfortunately, the version of tiledb used by the python interface is not up-to-date with the c++ tiledb library and it can only deal with  completely filled arrays.